# Fermispark Visualizations
## Author: David Sergio
## CSCD530 Big Data Analytics Project W2025
## This notebook reads binned photon count data and constructs an image from the data after it's been converted to a numpy array

In [9]:

import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
import matplotlib.pyplot as plt

In [10]:

f = 1

grid_precision = 0
bin_width_MeV = 10000
max_energy_MeV = 400000
filter_max_energy = True
filter_max_energy_bins = 20

file_name = f"../output_data/output_binned_counts_precision_{grid_precision}_binwidth_{bin_width_MeV}_400000.csv"

df = pd.read_csv(file_name, sep=',', usecols=range(0, filter_max_energy_bins))

print(df.shape)

(64383, 20)


In [11]:
360 * 180

64800

In [12]:


rows = 360 * (10 ** grid_precision)
cols = 180 * (10 ** grid_precision)

print(rows, cols, rows * cols)

print(f"precision_{grid_precision}_binwidth_{bin_width_MeV}_{max_energy_MeV}")

360 180 64800
precision_0_binwidth_10000_400000


## Convert RA, DEC, to 2D pixels for image generation:
### Currently working on implementation of a "Curve" feature. For now, it just stretches the image

In [13]:
def convert_RA_DEC_to_pixel(RA, DEC, curve = False):
    if curve:
        RA = (RA -0)  * np.cos(np.deg2rad(DEC))

    RA_pixel = int(RA * 10 ** grid_precision) + 3
    DEC_pixel = int((DEC + 90) * 10 ** grid_precision) + 3

    return RA_pixel, DEC_pixel

In [14]:


n_df_columns = len(df.columns)

if filter_max_energy:
    energy_buckets = filter_max_energy_bins - 2
    energy_bucket_groups = energy_buckets // 3
else:
    energy_buckets = n_df_columns - 2
    energy_bucket_groups = energy_buckets // 10

array = np.zeros((rows, cols, 4), dtype=np.uint8)
array[:, :, 3] = 255

print(array.shape)


(360, 180, 4)


## First make one pass through the data to find the maximum values for normalization
### Note: this is a work in progress, and there are several improvements that could be implemented here. This is only for demo purposes

In [15]:

red_max = 255
green_max = 255
blue_max = 255

red_max_val = 0
green_max_val = 0
blue_max_val = 0

nrows = 0

for index, row in df.iterrows():

    RA = row['RA']
    DEC = row['DEC']

    RA_pixel = int(RA * f) + 3
    DEC_pixel = int((DEC + 90) * f) + 3

    red = 0
    green = 0
    blue = 0

    row_sum = row.iloc[2:].sum()

    red = row.iloc[2:2 + energy_bucket_groups*1].sum()
    green = row.iloc[2 + energy_bucket_groups*1:2+energy_bucket_groups*2].sum()
    blue = row.iloc[2 + energy_bucket_groups*2:2+energy_bucket_groups*3].sum() 
    
    red_max_val = max(red_max_val, red)
    green_max_val = max(green_max_val, green)
    blue_max_val = max(blue_max_val, blue)

    nrows += 1

print(red_max_val, green_max_val, blue_max_val)


5023.0 9.0 3.0


## Convert the data to an image:

In [16]:

magnify = 1

interval = nrows // 10

for index, row in df.iterrows():

    RA = float(row['RA'])
    DEC = float(row['DEC'])

    


    RA_pixel, DEC_pixel = convert_RA_DEC_to_pixel(RA, DEC, curve=False)

    red = 0
    green = 0
    blue = 0

    row_sum = row.iloc[2:].sum()

    energy_bucket_groups = 1
    red = min(255, row.iloc[2:2 + energy_bucket_groups].sum() * magnify)
    green = min(255, row.iloc[2 + energy_bucket_groups*1:2+energy_bucket_groups*2 + 5].sum() * magnify)
    blue = min(255, row.iloc[2 + energy_bucket_groups*2 + 5:2+energy_bucket_groups*3].sum() * magnify)

    # red = int(red / red_max_val * red_max)
    # green = int(green / green_max_val * green_max)
    # blue = int(blue / blue_max_val * blue_max)

    alpha = 255

    red_radius = 1
    green_radius = 1
    blue_radius = 1
    alpha_radius = 1

    array[RA_pixel-red_radius:RA_pixel+red_radius, DEC_pixel-red_radius:DEC_pixel+red_radius, 0] = red

    array[RA_pixel-green_radius:RA_pixel+green_radius, DEC_pixel-green_radius:DEC_pixel+green_radius, 1] = green

    array[RA_pixel-blue_radius:RA_pixel+blue_radius, DEC_pixel-blue_radius:DEC_pixel+blue_radius, 2] = blue

    array[RA_pixel-alpha_radius:RA_pixel+alpha_radius, DEC_pixel-alpha_radius:DEC_pixel+alpha_radius, 3] = alpha

    # array[RA_pixel-3:RA_pixel+3, DEC_pixel-3:DEC_pixel+3] = [red, green, blue]
    

    if (index % interval == 0):
        print("index: ", index, "RGB: ", red, green, blue, "RA, DEC: ", RA, DEC, "RA_pixel, DEC_pixel: ", RA_pixel, DEC_pixel, "RGB MAx: ", red_max_val, green_max_val, blue_max_val)

# np.savetxt('gamma_image.csv', np.mean(array, axis=2), delimiter=',')
img = Image.fromarray(array)


center_x = convert_RA_DEC_to_pixel(128, -45)[0]
center_y = convert_RA_DEC_to_pixel(128, -45)[1]

radius = cols // 20

# The bounding box for the ellipse is [left, top, right, bottom].
bbox = [center_x - radius, center_y - radius, center_x + radius, center_y + radius]

draw = ImageDraw.Draw(img)
draw.ellipse(bbox, outline='white', width=1)

print(f"saved gamma_image_{grid_precision}_binwidth_{bin_width_MeV}_{max_energy_MeV}.png")
img.save(f"gamma_image_{grid_precision}_binwidth_{bin_width_MeV}_{max_energy_MeV}.png")
# img.show()

index:  0 RGB:  8.0 0.0 0.0 RA, DEC:  0.0 -89.0 RA_pixel, DEC_pixel:  3 4 RGB MAx:  5023.0 9.0 3.0
index:  6438 RGB:  169.0 1.0 0.0 RA, DEC:  321.0 -72.0 RA_pixel, DEC_pixel:  324 21 RGB MAx:  5023.0 9.0 3.0
index:  12876 RGB:  255 3.0 0.0 RA, DEC:  279.0 -54.0 RA_pixel, DEC_pixel:  282 39 RGB MAx:  5023.0 9.0 3.0
index:  19314 RGB:  53.0 0.0 0.0 RA, DEC:  237.0 -36.0 RA_pixel, DEC_pixel:  240 57 RGB MAx:  5023.0 9.0 3.0
index:  25752 RGB:  25.0 0.0 0.0 RA, DEC:  195.0 -18.0 RA_pixel, DEC_pixel:  198 75 RGB MAx:  5023.0 9.0 3.0
index:  32190 RGB:  23.0 0.0 0.0 RA, DEC:  153.0 0.0 RA_pixel, DEC_pixel:  156 93 RGB MAx:  5023.0 9.0 3.0
index:  38628 RGB:  3.0 0.0 0.0 RA, DEC:  114.0 18.0 RA_pixel, DEC_pixel:  117 111 RGB MAx:  5023.0 9.0 3.0
index:  45066 RGB:  46.0 0.0 0.0 RA, DEC:  72.0 36.0 RA_pixel, DEC_pixel:  75 129 RGB MAx:  5023.0 9.0 3.0
index:  51504 RGB:  23.0 0.0 0.0 RA, DEC:  30.0 54.0 RA_pixel, DEC_pixel:  33 147 RGB MAx:  5023.0 9.0 3.0
index:  57942 RGB:  43.0 1.0 0.0 RA, 